# TensorBoard 效能调校

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist


# 汇入 MNIST 手写阿拉伯数字 训练资料
(x_train, y_train),(x_test, y_test) = mnist.load_data()


# 训练/测试资料的 X/y 维度
x_train.shape, y_train.shape,x_test.shape, y_test.shape

# 特征缩放，使用常态化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0


In [2]:
# 建立、训练、评估模型
def train_test_model(hparams):
    # 建立模型
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    # 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    # 训练
    model.fit(x_train_norm, y_train, epochs=5, validation_split=0.2)
    # 评估，打分数
    _, accuracy = model.evaluate(x_test_norm, y_test, verbose=0)
    return accuracy

In [3]:
# 定义训练函数，包括依参数训练并写入 log
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)    # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar('accuracy', accuracy, step=1)

In [4]:
# 参数组合
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

In [5]:
# 依每一参数组合执行训练
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5311 - accuracy: 0.8415 - val_loss: 0.1603 - val_accuracy: 0.9563
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1749 - accuracy: 0.9491 - val_loss: 0.1163 - val_accuracy: 0.9660
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1242 - accuracy: 0.9641 - val_loss: 0.1107 - val_accuracy: 0.9666
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0983 - accuracy: 0.9698 - val_loss: 0.0895 - val_accuracy: 0.9737
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0765 - accuracy: 0.9772 - val_loss: 0.0971 - val_accuracy: 0.9708
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5310 - accuracy: 0.8462 - val_loss: 0.1550 - va

In [6]:
# 载入 TensorBoard notebook extension，即可在 jupyter notebook 启动 Tensorboard
%load_ext tensorboard

In [7]:
# 启动 Tensorboard
%tensorboard --logdir logs/hparam_tuning

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 4720.

## 使用浏览器输入以下网址，即可观看训练资讯：
## http://localhost:6006/

In [8]:
!taskkill /IM "tensorboard.exe" /F
# or 
# !taskkill /F /PID 15784

成功: 處理程序 "tensorboard.exe" (PID 4720) 已經終止了。
